## Option 1: BERT processing

In [1]:
from bert_serving.client import BertClient
import pandas as pd

In [2]:
desc=pd.read_csv('data/bbg_chn_intro_processed.csv',index_col=0)
desc

,company,intro,length
0,600000.SH,上海浦东发展银行股份有限公司提供银行服务。该公司提供贷款、存款、账户结算、外汇和 其他服务。...,66
1,600004.SH,广州白云国际机场股份有限公司提供机场辅助服务。该公司提供地面清洁、飞机保养、飞机 维修和其他...,80
2,600006.SH,东风汽车股份有限公司设计、生产和销售柴油发动机、轻型卡车、铸件和相关零部件。该公 司的产品使...,54
3,600007.SH,中国国际贸易中心股份有限公司提供房地产服务。该公司经营办公楼租赁业务、购物商城租 赁业务、公...,83
4,600008.SH,北京首都创业股份有限公司提供供水服务。该公司提供水输送、供水系统管理、供水网络维 护等服务。...,70
...,...,...,...
3815,300830.SZ,金现代信息产业股份有限公司开发软件产品。该公司生产数据可视化软件、移动应用软件、 测试软件、...,98
3816,300831.SZ,西安派瑞功率半导体变流技术股份有限公司研究、开发、生产和销售电力半导体元件设备。 该公司生产...,84
3817,300832.SZ,深圳市新产业生物医学工程股份有限公司是一家生物医学公司。该公司研究、开发、生产和 销售临床实...,80
3818,300833.SZ,广州市浩洋电子股份有限公司生产和销售照明装置。该公司生产表演艺术照明装置、建筑和 商业照明装...,76


In [15]:
bc = BertClient('10.0.0.1')
embedding = bc.encode(desc.intro.values.tolist())

In [16]:
embedding_cbd=desc[['company']].reset_index(drop=True).merge(pd.DataFrame(embedding),how='left',left_index=True,right_index=True)
embedding_cbd.head()

,company,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,600000.SH,-0.483388,0.212476,-0.153359,-0.110177,0.365436,0.261677,-0.412337,0.453939,-0.232221,...,-0.029137,-0.459134,-0.306352,0.529452,0.234407,-0.378056,-0.134923,-0.105652,0.529955,-0.013904
1,600004.SH,-0.330700,0.215865,0.337870,0.111086,0.181045,0.308193,-0.269856,0.417020,-0.150681,...,-0.296666,-0.373251,-0.359413,0.193572,0.309502,-0.410172,-0.201505,-0.131555,0.040491,0.094853
2,600006.SH,-0.209895,0.144942,0.228274,0.377867,0.041713,0.291694,-0.341646,0.366442,-0.267444,...,0.121361,-0.133171,-0.386811,0.176484,0.108471,-0.003087,0.095810,-0.461561,0.261568,0.200326
3,600007.SH,-0.567843,0.188588,0.260582,0.249165,0.544975,0.244919,-0.173271,0.445236,-0.070529,...,-0.150018,-0.366114,-0.635301,0.667324,0.440636,-0.473521,-0.021758,-0.046649,0.082277,0.027114
4,600008.SH,-0.298855,0.129970,0.154596,0.208546,0.161277,0.397346,-0.264920,0.602614,-0.227778,...,-0.165707,-0.294478,-0.405932,0.322581,0.097261,-0.241579,-0.129864,0.028017,0.198446,0.080833


In [17]:
embedding_cbd.to_csv("bbg_chn_bert.csv")

## Option 2: Sentence BERT processing

In [1]:
from sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging
import pandas as pd

In [12]:
desc = pd.read_csv("data/bbg_eng_lemma.csv", index_col=0)
sentences = desc["intro_lemmatized"].values.tolist()

In [13]:
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)
model = SentenceTransformer("../US_market/bert-large-nli-stsb-mean-tokens.zip")
sentence_embeddings = model.encode(sentences)

2020-06-02 12:45:23 - Load pretrained SentenceTransformer: ../US_market/bert-large-nli-stsb-mean-tokens.zip
2020-06-02 12:45:23 - Load SentenceTransformer from folder: ../US_market/bert-large-nli-stsb-mean-tokens.zip
2020-06-02 12:45:23 - loading configuration file ../US_market/bert-large-nli-stsb-mean-tokens.zip/0_BERT/config.json
2020-06-02 12:45:23 - Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty":

Batches: 100%|██████████| 478/478 [00:27<00:00, 17.29it/s]


In [14]:
desc = desc.merge(
    pd.DataFrame({"bert_vec": sentence_embeddings}), left_index=True, right_index=True
)
desc.bert_vec = desc.bert_vec.apply(list)
desc.head()

,company,intro,introLen,lemma,intro_lemmatized,bert_vec
0,600000.SH,"Shanghai Pudong Development Bank Co., Ltd. pro...",266,"['shanghai', 'pudong', 'development', 'bank', ...",shanghai pudong development bank co ltd provid...,"[-1.3002316, -1.1620108, -0.526951, -0.6841726..."
1,600004.SH,"Guangzhou Baiyun International Airport Co.,Ltd...",290,"['guangzhou', 'baiyun', 'international', 'airp...",guangzhou baiyun international airport co ltd ...,"[-0.14394785, -0.36425644, 0.34150198, -0.5029..."
2,600006.SH,"Dongfeng Automobile Co., Ltd. designs, manufac...",129,"['dongfeng', 'automobile', 'co', 'ltd', 'desig...",dongfeng automobile co ltd design manufacture ...,"[-0.63153875, -0.17034253, 0.006435485, -0.399..."
3,600007.SH,"China World Trade Center Co., Ltd. provides re...",288,"['china', 'world', 'trade', 'center', 'co', 'l...",china world trade center co ltd provide real e...,"[-0.44283512, 0.093182474, -0.0055609215, -1.3..."
4,600008.SH,"Beijing Capital Co.,Ltd offers water supply se...",270,"['beijing', 'capital', 'co', 'ltd', 'offer', '...",beijing capital co ltd offer water supply serv...,"[-0.36592022, 0.09057382, 0.31063876, -0.67287..."


In [16]:
desc.to_csv("bert_output_bbg_eng.csv")

In [15]:
len(sentence_embeddings[0])

1024